# 聚类简介

聚类是将样本集合中相似的样本(实例)，依据它们特征的相似度或距离，分配到相同的类，不相似的样本分配到不同的类。

聚类分硬聚类和软聚类。

- 硬聚类

  每一个样本属于某一类，找到一个模型$z = g_\theta(x)$ 。

- 软聚类

  每一个样本依概率属于每一个类,找到一个条件概率分布$P_\theta(z|x)$ ,$x$是样本向量，$z$是类别。

聚类方法有层次聚类和k均值聚类。





# k-means

## 原理

k均值聚类是基于中心的聚类方法， 通过迭代，将样本分到k个类中，使得每个样本与其所属类的中心或均值最近; 得到k个“ 平坦的”、非层次化的类别，构成对空间的划分 。

k均值聚类的目标是将n个样本分到 k个不同的类或簇中，这里假设k<n。

k个类$G_1,G_2...G_k$对$X$，进行划分，其中$G_i\cap G_j=\varnothing$，$\bigcup_{i=1}^kG_i=X$。用C表示划分，一个划分代表一个聚类结果。C是一个一对多的函数 $l=C(i),i\in\{1,2,3...n\},l\in\{1,2,3...k\}$。

所以k均值聚类模型是一个从样本到类的函数。k均值聚类的策略是通过损失函数的最小化选取最优的划分或函数$C^*$。

采用欧式距离的平方：
$$
d(x_i,x_j)=\sum_{k=1}^m(x_{ki}-x{kj})^2\\
=||x_i-x_j||^2
$$
定义样本与其所属类的中心之间的距离的总和为损失函数：
$$
W(C)= \sum_{l=1}^k \sum_{C(i)=l}||x_i-\bar{x}_l||^2
$$
这里 $\sum_{C(i)=l}$ 代表限制了分到所属类的样本才计算距离，外面一层$\sum_{l=1}^k$计遍历算出每一个类他的样本和其中心距离的总和。$W(C)$就代表了同类中样本的相似程度。

k均值就是求解最优化问题
$$
C^* = argmin_C W(C)\\
 =argmin_C \sum_{l=1}^k \sum_{C(i)=l}||x_i-\bar{x}_l||^2
$$
相似的样本被聚到同类时，损失西数值最小。



## 算法

输入：$n$个样本集合$X$

输出：样本集合的聚类$C^*$

1.**初始化**，$t=0$，随机选择k个样本点作为聚类中心$m^{(0)}=(m_1^{(0)},...,m_l^{(0)},...m_k^{(0)})$。

2.**对样本进行聚类**，对固定类中心$m^{(t)}=(m_1^{(t)},...,m_l^{(t)},...m_k^{(t)})$，其中$m_l^{(t)}$ 是类$G_l$的中心，聚酸每个样本到类中心的距离，将每个样本指派到与其最近的中心的类中，构成聚类结果$C^{(t)}$。

3.**计算新的类中心**，对聚类结果$C^{(t)}$，计算当前各类中的样本均值，作为新的类中心，$m^{(t+1)}=(m_1^{(t+1)},...,m_l^{(t+1)},...m_k^{(t+1)})$

4.**迭代符合条件**，输出$C^*=C^{(t)}$。否则$t=t+1$，返回（2）

复杂度是$O(mnk$），$m$是样本维数，$n$是样本个数，$k$是类的个数。



**特点**

基于划分的聚类方法:类别数k事先指定 ;以欧氏距离平方表示样本之间的距离，以中心或样本的均值表示类别;以样本和其所属类的中心之间的距离的总和为最优化的目标函数:得到的类别是平坦的、非层次化的:算法是迭代算法，不能保证得到全局最优。



k均值聚类属于启发式方法，不能保证收敛到全局最优，**初始中心的选择会直接影响聚类结果，选择不同的初始中心 ，会得到不同的聚类结果**。注意，类中心在聚类的过程中会发生移动，但是往往不会移动太大，因为在每一步，样本被分到与其最近的中心的类中。



k均值聚类中的类别数人值需要预先指定，而在实际应用中最优的么值是不知道的。解决这个问题的一个方法是尝试用不同的k值聚类，检验各自得到聚类结果的质量，推测最优的k值。 **聚类结果的质量可以用类的平均直径来衡量。一般地，类别数变小时，平均直径会增加:类别数变大超过某个值以后，平均直径会不变:而这个值正是最优的k值**。



**==当集群具有不同的大小、不同的密度或非球形时，K-Means算法的性能不是很好==**，因为将实例分配给某个集群时，它所关心的只是与中心点的距离。**在这种类型的椭圆簇上，高斯混合模型非常有效。**

## 程序实现

```python
from sklearn.cluster import KMeans
k= 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)
```



查看k个中心点位置

```python
kmeans.cluster_centers_
```



用新数据进行聚类

```Python
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
kmeans.predict(X_new)
```



## 寻找最优解

**通过给定初始值**

因为最后聚类结果和一开始的初始化中心点有关，如果能一开始就知道大致位置，那就可以更容易收敛到正确解

```Python
good_init = np.array([[-3, 3], [-3, 2], [-3, 1], [-1, 2], [0, 2]])
kmeans = KMeans(n_clusters=5, init=good_init, n_init=1)
```



**通过多组，选择最优**

使用不同的随机初始化多次运行算法，并保留最 优解。随机初始化的数量由超参数n_init控制:默认情况下，它等于 10，这意味着当你调用fit()时，前述算法运行10次，Scikit-Learn 会保留最优解。**通过模型的惯性，即每个实例与其最接近的中心点之间的均方距离**来判断是否是最优解。KMeans类运行n_init次算法，并保留模型的最低惯性。



查看模型的惯性

```Python
kmeans.inertia_
```





## 模型的改进

**K-Means++**

优化了模型的初始化步骤，该算法倾向于选择彼此相距较远的中心点，这一改进使得K-Means算法收敛到次优解的可能性很小

**步骤**

1.从数据集随机选取一个中心点$c^{(1)}$。

2.选取一个新的中心点$c^{(i)}$，选择一个概率是$D(x^{(i)})^2/\sum_{j=1}^m D(x^{(j)})^2$，这里的$D(x^{(i)})$是实例$x^{(i)}$与已经选择的最远中心点的距离，这种概率分布确保距离已选择的中心点较远的实例有更大可能性被选择为中心点。

3.重复上述步骤，直至选择了k个中心点。

默认情况下，KMeans类使用这种K-Means++初始化方法。除非将init变成random才是传统的。





**MiniBatchKMeans**

该算法能够在每次迭代中使用小批量K-Means稍微移动中 心点，而不是在每次迭代中使用完整的数据集。这将算法的速度提高了 3到4倍，并且可以对不容纳内存的大数据集进行聚类。

```Python
from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans = MiniBatchKMeans(n_clusters=5)
minibatch_kmeans.fit(X)
```

**缺点**

尽管小批量K-Means算法比常规K-Means算法快得多，但其惯性通常略差一些，尤其是随着集群的增加。





**寻找最佳聚类数**

尝试选择k时，惯性不是一个好的性能指标，因为 随着k的增加，惯性会不断降低。实际上，集群越多，每个实例将越接 近其最接近的中心点，因此惯性将越低。但我们也可以画一个k与惯性的图，可以看出随着k增大到4，惯性下降得很快，但随着k继续增大， 惯性下降得很慢，所以可以选择4。

但这个方法过于粗糙，可以使用轮廓分数来描述，它是所有实例的平均轮廓系数。

实例的轮廓系数等于(b-a)/max(a，b)，其中a是与同一集群中其他实例 的平均距离(即集群内平均距离)，b是平均最近集群距离。轮廓系数可以在-1和+1之间变化。接近+1的系数表示该实例 很好地位于其自身的集群中，并且远离其他集群，而接近0的系数表示 该实例接近一个集群的边界，最后一个接近-1的系数意味着该实例可能 已分配给错误的集群。

```Python
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)
```



# 高斯混合模型



## 原理



有几种GMM变体。在GaussianMixture类中实现的最简单变体中，必须事先知道高斯分布的数量k。假定数据集X是通过以下概率过程生成的:

- 对于每一个实例，从k个集群中随机选择一个集群。选择第j个集群的概率由集群的权重$\Phi^{(j)}$定义，第i个实例选择的集群索引记作$z^{(i)}$。

- 如果$z^{(i)}=j$，就表示第i个实例分配给了第j个集群，则从高斯分布中随机采样该实例的位置$x^{(i)}$。其中均值$\mu^{(j)}$和协方差矩阵$\sum^{(j)}$。$x^{(i)} ~N(\mu^{(j)},\sum^{(j)})$。







## 程序实现

给定数据集X，你通常希望首先估计权重$\Phi$以及所有分布参数$\mu^{(1)}$至$\mu^{(k)}$和$\sum^{(1)}$至 $\sum^{(k)}$。

```Python
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=3, n_init=10)
gm.fit(X)



gm.weights_

gm.means_

gm.covariances_

```





# 层次聚类

层次聚类又有聚合 (自下而上)和分裂 (自上而下)两种方法。

- 聚合法开始将每个样本各自分到一个类;之后将相距最近的两类合并，建立一个新的类，重复此操作直到满足停止条件:得到层次化的类别。

- 分裂法开始将所有样本分到一个类;之后将己有类中相距最远的样本分到两个新的类，重复 此操作直到满足停止条件;得到层次化的类别。





# 距离

聚类的核心概念是相度或距离，有多种相似距离的定义。因为相似度直接影响聚类的结果，所以其选择是聚类的根本问题。具体哪种 相似度更合适取决于应用问题的特性。距离越大相似度越小，距离越小相似度越大。



样本$X$，$X=[x_{ij}]_{m\times n}$，$x_{ij}$代表第$j$个样品的第$i$个属性。

$x_i=(x_{1i},x_{2i}...x_{mi})$，$x_j=(x_{1j},x_{2j}...x_{mj})$



**闵可夫斯基距离**

样本$x_i$和$x_j$的闵可夫斯基的距离定义为：
$$
d_{ij} = (\sum_{k=1}^m|x_{ki}-x_{kj}|^p)^{\frac{1}{p}}
$$
当$p=2$是欧氏距离，$p=1$是曼哈顿距离，$p=\infin$是切比雪夫距离（取各个坐标数值差的绝对值的最大值）。



**马哈拉诺比斯距离**

给定一个样本集合$X$，$X=[x_{ij}]_{m\times n}$，其协方差短阵记作$S$。样本$x_i$和$x_j$的马哈拉诺比斯距离：
$$
d_{ij} = [(x_i-x_j)^TS^{-1}(x_i-x_j)]^{\frac{1}{2}}
$$
当$S$为单位矩阵时，即样本数据的各个分量互相独立且各个分量的方差为1时，由式知马氏距离就是欧氏距离，所以马氏距离是欧氏距离的推广。



